In [117]:
%run food_item_info.ipynb
%run evaluation_notebook.ipynb
import random
types = ['Pantry', 'Refrigerate', 'Freeze']

nlp = spacy.load(MODEL_PATH)
allFoodTweets = []
rangeTweets = [5,7]
for i in range(rangeTweets[0], rangeTweets[1]):
    myTweet = test_data['tweet'][i]
    x = nlp(preProcess(myTweet))
    # splitSent = myTweet.split()
    allFoodTweets.append(x.doc.ents)

counter = rangeTweets[0]
for entities in allFoodTweets:
    if counter != rangeTweets[1]:
        myTweet = test_data['tweet'][counter]
        ent_recognize(preProcess(myTweet))
        print("Total Entities found: ", entities)
        counter += 1
    for e in entities:
        item = repr(e)
        print(item)
        if entityFound(item):
            for t in types: print(foodStorage(item, t))
        else: print('entity not found!')

# displayTipsByTweet(test_data)

Total Entities found:  (eggs, ham, bacon bits, cream cheese, honey)
eggs
['No Tip to give!']
['The Date of Perish for Eggs is from 3 to 5 Weeks']
['Freeze is Not Recommended']
ham
['No Tip to give!']
['The Date of Perish for Ham is from 6 to 9 Months']
['Freeze is Not Recommended']
bacon bits
['The Date of Perish for Bacon bits is 1 Years']
['Refrigerate is Package use-by date']
['No Tip to give!']
cream cheese
['No Tip to give!']
['The Date of Perish for Cream cheese is 2 Weeks']
['Freeze is Not Recommended']
honey
['The Date of Perish for Honey is 2 Years']
['No Tip to give!']
['No Tip to give!']


Total Entities found:  (potatoes, tomato salad, cheese, lamb)
potatoes
['A tip about Potatoes is that This is the recommended storage method for whole potatoes.', 'The Date of Perish for Potatoes is from 1 to 2 Months']
['A tip about Potatoes is that The United States Potato Board notes refrigeration can cause potatoes to darken during cooking and to develop an unpleasantly sweet taste.', 'The Date of Perish for Potatoes is from 1 to 2 Weeks']
['Potatoes can be stored in the Freeze for 10 to 12 Months', 'A tip about Potatoes is that Freezer timeline applies to cooked and mashed potatoes.']
tomato salad
entity not found!
cheese
['No Tip to give!']
['The Date of Perish for Cheese is 6 Months', 'Cheese can be stored after opened in the Refrigerate for 3 to 4 Weeks']
['The Date of Perish for Cheese is 6 Months']
lamb
['No Tip to give!']
['The Date of Perish for Lamb is from 3 to 5 Days']
['The Date of Perish for Lamb is from 4 to 12 Months']


In [107]:
import pandas as pd

LIVE_TWITTER_DATA_PATH = "eco1friendly.csv"

recent_tweets = pd.read_csv(LIVE_TWITTER_DATA_PATH)
recent_tweets.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1509622830111019009,1509622830111019009,2022-03-31 13:05:04 Pacific Daylight Time,2022-03-31,13:05:04,-700,1493744544420007936,fiskeratiblog,Fiskerati,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1509621829861793796,1509621829861793796,2022-03-31 13:01:06 Pacific Daylight Time,2022-03-31,13:01:06,-700,10286782,earth911,Earth911.com,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1509621757380018195,1509621757380018195,2022-03-31 13:00:49 Pacific Daylight Time,2022-03-31,13:00:49,-700,1234114342200762368,khsrc1,Koala History and Sustainability Research Cluster,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1509620267961577473,1509620267961577473,2022-03-31 12:54:53 Pacific Daylight Time,2022-03-31,12:54:53,-700,18028541,biofriendly,Biofriendly,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1509619486122520587,1509619486122520587,2022-03-31 12:51:47 Pacific Daylight Time,2022-03-31,12:51:47,-700,223164111,atena1978,Sara Montero Dueñas,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [118]:
new_data = recent_tweets['tweet']
new_data.head()


0    Fisker’s Latest Visit to Magna Steyr Shows CO2...
1    During my childhood, when someone in the famil...
2    Thank you, Gabi, for pointing this crowd out. ...
3    Why Kids (and Parents) Need to Spend More Time...
4    Hoy os traigo una novedad #ecofriendly. Discos...
Name: tweet, dtype: object

In [137]:
import re
def preProcessLocal(tweet):
    #Converts a tweet to lowercase, replaces anyusername w/ <USERNAME> and URLS with <URL>
    tweet = tweet.lower()
    tweet = re.sub('@[a-zA-z0-9]*', '', tweet)              # <USERNAME>
    tweet = re.sub('http[a-zA-z0-9./:]*', '', tweet)       # <URL>
    tweet = re.sub('[.,-]*', '', tweet)
    
    # Utilize for instragram posts, remove hashtag for food-related posts
    tweet = re.sub(r'#[a-zA-z0-9]*', '', tweet)
    # tweet = re.sub(r'#', '', tweet) 
    
    tweet = re.sub('&amp;', 'and', tweet)
    return tweet
# print(preProcessLocal(recent_tweets['tweet'][0]))

In [142]:
nlp = spacy.load(MODEL_PATH)
allFoodTweets = []
rangeTweets = [22,28]
for i in range(rangeTweets[0], rangeTweets[1]):
    myTweet = recent_tweets['tweet'][i]
    x = nlp(preProcessLocal(myTweet))
    allFoodTweets.append(x.doc.ents)

counter = rangeTweets[0]
for entities in allFoodTweets:
    if counter != rangeTweets[1]:
        myTweet = recent_tweets['tweet'][counter]
        ent_recognize(preProcessLocal(myTweet))
        print("Total Entities found: ", entities)
        counter += 1
    for e in entities:
        item = repr(e)
        print(item)
        if entityFound(item):
            for t in types: print(foodStorage(item, t))
        else: print('entity not found!')

Total Entities found:  (milk, bread)
milk
['No Tip to give!']
['Refrigerate is Package use-by date']
['The Date of Perish for Milk is 3 Months']
bread
['Bread can be stored in the Pantry for 3 to 5 Days', 'A tip about Bread is that May have shorter shelf life due to lack of preservatives.']
['Bread can be stored in the Refrigerate for 2 to 3 Months']
['No Tip to give!']


Total Entities found:  ()


Total Entities found:  ()


Total Entities found:  ()


Total Entities found:  ()


Total Entities found:  (butter,)
butter
['A tip about Butter is that May be left at room temperature for 1 - 2 days.']
['The Date of Perish for Butter is from 1 to 2 Months']
['The Date of Perish for Butter is from 6 to 9 Months']
